# re Module

---
doc: https://docs.python.org/3/library/re.html

In [1]:
import re

## Raw Strings

Strings cruas como diz a tradução literal são strings que não contêm caracteres especiais como `\n` ou `\t`. Isso facilita na análise de strings.

## Compile

A maneira mais eficiente de realizar múltiplas busca em um arquivo de texto atrás de um padrão, é mandar a biblioteca `re` compilar o nosso padrão e guardar em um objeto `re.compile(pattern)`

## Finditer

Vamos agora procurar todos os *matches* entre o padrão passado e o texto. Para isso vamos realizar a seguinte operação

```python
text = ... # Uma string representando um texto
pattern = re.compile(r'...')
matches = pattern.finditer(text)
```

`matches` será um objeto definido por ser *iterável* (`callable_iterator object`). Podemos acessar cada match usando um `for match_found in matches` de modo que cada `match_found` é um outro objeto chamado `SRE_Match`. Este objeto por sua vez tem os atributos:

* `span` = onde está localizada a string no texto, para obter acesse em forma de tupla `match_found.span()`
* `match` = a string encontrada

Portanto, se eu quiser obter a string `match` do texto original eu faria

```python
for match_found in matches:
    ini = match_found.span()[0]
    end = match_found.span()[1]
    print( text[ini:end] )
```

## Procurando por caracteres especiais

Muitos caracteres especiais são usados pelo regex para simbolizar algo. Por exemplo, o caracter `.` dá match com todo caracter (exceto com o carácter `\n` se a tag `DOTALL` estiver desativada).

Para encontrar especificamente o caracter `.` eu tenho que usar no padrão a contrabarra `re.compile(r'\.')`

## Caracteres especiais na biblioteca

|Caracter|Significado|
| :-: |-|
|`.`| Qualquer caracter|
|`\d`| Dígito (0-9)|
|`\w`| Caracter de palavras (a-z, A-Z, 0-9, \_)|
|`\s`| Espaço vazio (' ', '\t', '\n')|

Esses mesmos caracteres com as letras em caixa alta selecionam o oposto. Por exemplo, `\D` seleciona tudo que não é um dígito.

## Âncoras

Não selecionam um caracter em específico, mas selecionam uma condição de contorno para o padrão (algo que tem que vir antes, ou algo que tem que vir depois).

|Caracter|Significado|
| :-: |--------------------|
|`\b`| Borda de *word*|
|`^`| Começo da String|
|`$`| Fim da string|

Usando $` no final do padrão vai retornar somente o caso colado com o final, por exemplo.

## Conjunto de Caracteres

Para indicar conjunto de caracteres usamos os colchetes e dentro colocamos todos os caracteres que desejamos. Por exemplo, se queremos selecionar somente a letra `f` ou `z` escreveríamos `re.compile(r'[fz]')`. Mas atenção, essa seleção só seleciona *um único caracter*, vai devolver todas as aparições de `f` e `z` no texto.

Se colocarmos dentro do conjunto o caracter `^`, isso equivale a estar negando o conjunto. Por exemplo, `[^123]` representa todos os caracteres, exceto 1, 2 e 3.

## Alcance de Caracteres

O traço pode representar um alcance de caracteres. Vamos supor que queremos somente números entre 4 e 8, então colocaremos no conjunto o `[4-8]`. Outro exemplo é: vamos supor que queremos representar somente as letras que aparecem na representação hexadecimal (não importa se é maiúscula ou minúscula), então teríamos: `[a-fA-F]`

## Quantificadores

* {m} - contador com exatamente *m*
* {m,n} - Contador *guloso* que vai encontrar de *m* até *n* repetições do que for marcado.

Omitir *m* diz que o limite inferior é 0. Omitir *n* diz que o limite superior é infinito. 

Esse contador é considerado *guloso*, pois ele vai considerar o maior número de repetições.

Abreviações de Contadores

* `*` - Seleciona 0 ou infinitos == `{,}`
* `+` - Seleciona 1 ou infinitos == `{1,}`
* `?` - Seleciona 0 ou 1 == `{,1}`

Todo quantificador, quando sucedido por `?` se reduz ao menor caso. Isto é, a versão não *golusa* do quantificador.

## Grupos

Grupos são representados por *parenteses* `()`, isto é, podemos colocar diversos padrões opcionais separados pela barra vertical `|` que indica o operador `ou`. O resultado seria algo como: `(desse_jeito|ou_desse|ate_esse)`

Podemos ainda usar grupos para coletar informações diretamente do que foi comparado. Por exemplo, uma expressao como `alguma(expressao)(interessante)`, podemos acessar o conteudo de cada grupo com auxilio do metodo `.group(i)` do objeto de `Match`. Onde `i = 0` indica toda a expressao, `i = 1` indica o primeiro grupo da esquerda para direita e `i = 2` o segundo.

## Appendix

### Exemplo Substituindo Negritos, Itálicos

```python
texto = "*Nun64* et sem* eget ***AR+C-U*** rutrum ornare** *tipo de **pardal** et* id **lorem**, ou ainda, *caraca **bixo***. **FIM**"

# Grupo 1: (\s|^) Espaço em branco OU começo de string
# \*{N} numero N de asteriscos
# Grupo 2: (.+?) qualquer caracter com 1 ou mais (de menor tamanho possivel)
# \*{N} numero N de asteriscos
negrito_it = re.compile(r'(\s|^)\*{3}(.+?)\*{3}')
negrito = re.compile(r'(\s|^)\*{2}(.+?)\*{2}')
italico = re.compile(r'(\s|^)\*{1}(.+?)\*{1}')


novo_texto = negrito_it.sub(r'\1<strongemph>\2</strongemph>', texto)
novo_texto = negrito.sub(r'\1<strong>\2</strong>', novo_texto)
novo_texto = italico.sub(r'\1<emph>\2</emph>', novo_texto)

print(novo_texto)
```

SAÍDA: 

```
<emph>Nun64</emph> et sem* eget <strongemph>AR+C-U</strongemph> rutrum ornare** <emph>tipo de <strong>pardal</strong> et</emph> id <strong>lorem</strong>, ou ainda, <emph>caraca <strong>bixo</strong></emph>. <strong>FIM</strong>
```

### Exemplo IDS de vídeos no youtube

```python
# Context: Will YouTube Ever Run Out Of Video IDs? [https://youtu.be/gocwRvLhDf8]

texto = "https://www.youtube.com/watch?v=K8L6KVGG-7o\nhttps://www.youtube.com/watch?v=YYXdXT2l-Gg&list=PL-osiE80TeTt2d9bfVyTiXJA-UTHn6WwU"

# Quero extrair o ID desses vídeos : K8L6KVGG-7o e YYXdXT2l-Gg
# Os IDS do youtube usam base 64 com ultimos digitos sendo "_" e "-"

# < v= > exatamente isso
# < [\w-]+ > 1 ou mais caracteres, no conjunto "word" (= [a-zA-Z0-9_]) e "-"
padrao_id = re.compile(r'v=([\w-]+)')

padroes_encontrados = padrao_id.finditer(texto)

for padrao_encontrado in padroes_encontrados:
    print(padrao_encontrado.group(1))
```

SAÍDA:

```
K8L6KVGG-7o
YYXdXT2l-Gg
```

### Exemplo encontrando emails válidos

```python
texto = '''bol@uol.com.br
abc.game@email.net
OtherMail123@strange-mail.org
institutional-account@university.edu.br
notaanemail@ola
notanemail2@ponto.com.
'''

# < [\w.-]+ > um conjunto de caracteres word \w (=[a-zA-Z0-9_]), ponto e traço ocorrendo 1 ou mais vezes 
# < @ > obrigatorio
# < [a-z\-]+ > conjunto de caracteres minusculos de a até z ou um traço, ocorrendo 1 ou mais vezes
# < \.(com|edu|net|org) > um ponto obrigatorio seguido de com OU edu OU net OU org
# (\s|\.[a-z]{2}) um espaco vazio OU um ponto seguido de 2 letras minusculas
padrao_email = re.compile(r'[\w.-]+@[a-z\-]+\.(com|edu|net|org)(\s|\.[a-z]{2})')

padroes_encontrados = padrao_email.finditer(texto)

for padrao_encontrado in padroes_encontrados:
    print(padrao_encontrado)
```

SAÍDA:

```
<re.Match object; span=(0, 14), match='bol@uol.com.br'>
<re.Match object; span=(15, 34), match='abc.game@email.net\n'>
<re.Match object; span=(34, 64), match='OtherMail123@strange-mail.org\n'>
<re.Match object; span=(64, 103), match='institutional-account@university.edu.br'>
```


### Exemplo do prefixo de senhor e senhora

```python
texto = "Mr. Incrivel\nMr Incrivel\nMister Incrivel\nMs Simpson\nMiss Pig\nMrs. Sanders\nMrsa Vodka"

# Quero encontrar todos os nomes que comecam com Mr
# Para isso vou usar o quantificador "?" que indica um caracter opcional

# < M > exatamente isso
# < (r|s|rs) > pode ser: r OU s OU rs 
# < \.? > tem um ponto opcional
# < \s > tem um espaco
# <.*> quantos caracteres seguirem até o fim da linha
padrao = re.compile(r'M(r|s|rs)\.?\s.*')

padroes_encontrados = padrao.finditer(texto)

for padrao_encontrado in padroes_encontrados:
    print(padrao_encontrado)
```

SAÍDA:

```
<re.Match object; span=(0, 12), match='Mr. Incrivel'>
<re.Match object; span=(13, 24), match='Mr Incrivel'>
<re.Match object; span=(41, 51), match='Ms Simpson'>
<re.Match object; span=(61, 73), match='Mrs. Sanders'>
```

### Exemplo filtrando nomes por DDD

```python
texto = 'Lista telefonica:\nAna - (11)1234-5678\nBeatriz - (11)4312-1223\nCamila - (19)3367-1234\nDebora - (15)2336-9972\nEloisa - (11)9982-2230'

# Dividir a lista em linhas
linhas = texto.split('\n')

# < \(11\) > Inicia com 11 entre parenteses,
# < \d{4} > tem 4 digitos,
# < \- > depois tem um traco,
# < \d{4} > depois mais 4 digitos.
padrao_telefone = re.compile(r'\(11\)\d{4}\-\d{4}') 

# <^> Começo da string, 
# <\w> caracteres de palavra,
# <+> com  de 1 ou mais, 
# <\s> até o espaço.
padrao_nome = re.compile(r'^\w+\s') 

# Analisar cada linha
for linha in linhas:
    
    # Procura se tem (11) seguido de um telefone.
    match_regiao = re.search(padrao_telefone, linha)
    
    # Se M for None -> (not M) == True
    if not match_regiao:
        continue
    else:
        # Significa que encontrou telefone com DDD (11)
        # Agora vou coletar o nome
        match_nome = re.search(padrao_nome, linha)
        if match_nome:
            comeco = match_nome.span()[0]
            fim = match_nome.span()[1] - 1 # O fator -1 desconsidera o espaço vazio
            print(linha[comeco:fim])
```

SAÍDA:

```
Ana
Beatriz
Eloisa
```

### Exemplos de Número de Celular

```python
texto = '235-555-0042 outros numeros 1231231241241 e coisas como 235-555-abcd\nPosso ter 235-5550042 ou ainda 235-555-004 somente.\nMas eu gosto quando tenho algo como 123-456-7890, mas tambem nao tem pr0blem4 se usar ponto 420.420.0420. Só não gosto disso quando tiver estrela 333*111*4444'

p1 = re.compile(r'\d{3}[-.]\d{3}[-.]\d{4}')
M = p1.finditer(texto)

for m in M:
    print(m)
```

SAÍDA: 

```
<re.Match object; span=(0, 12), match='235-555-0042'>
<re.Match object; span=(157, 169), match='123-456-7890'>
<re.Match object; span=(213, 225), match='420.420.0420'>
```

### Exemplos de Âncoras

* Usando `^`

```python
texto = 'O rato roeu a roupa do Rei de ROMa'

p1 = re.compile(r'O')
p2 = re.compile(r'^O')

M1 = p1.finditer(texto)
M2 = p2.finditer(texto)

for m1 in M1:
    print(m1)

print('-'*40)
for m2 in M2:
    print(m2)
```

SAÍDA:

```
<re.Match object; span=(0, 1), match='O'>
<re.Match object; span=(31, 32), match='O'>
----------------------------------------
<re.Match object; span=(0, 1), match='O'>
```

* Usando `$`

```python
texto = 'O rato roeu a roupa do Rei de ROMa'

p1 = re.compile(r'a')
p2 = re.compile(r'a$')

M1 = p1.finditer(texto)
M2 = p2.finditer(texto)

for m1 in M1:
    print(m1)

print('-'*40)
for m2 in M2:
    print(m2)
```

SAÍDA

```
<re.Match object; span=(3, 4), match='a'>
<re.Match object; span=(12, 13), match='a'>
<re.Match object; span=(18, 19), match='a'>
<re.Match object; span=(33, 34), match='a'>
----------------------------------------
<re.Match object; span=(33, 34), match='a'>
```

* Usando `\b`

```python
texto = 'O rato roeu a roupa do Rei de ROMa'

p1 = re.compile(r'O')
p2 = re.compile(r'O\b')

M1 = p1.finditer(texto)
M2 = p2.finditer(texto)

for m1 in M1:
    print(m1)

print('-'*40)
for m2 in M2:
    print(m2)
```

SAÍDA

```
<re.Match object; span=(0, 1), match='O'>
<re.Match object; span=(31, 32), match='O'>
----------------------------------------
<re.Match object; span=(0, 1), match='O'>
```

In [ ]:
# Uma demosntração rápida de operadores ternários

x = 1

if x == 1:
    print('x é 1')
else:
    print('x não é 1')
    
print('x é 1') if (x == 1) else print('x não é 1')
    
# JS
# (condition) ? [if true] : [if false]
# (x == 1) ? console.log('x é 1') : console.log('x não é 1')

# Python
# print('x é 1') if (x == 1) else print('x não é 1')